In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-15 00:03:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-11-15 00:03:54 (8.54 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NCAA").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Extraction

In [ ]:
from pyspark import SparkFiles
# Connecting to S3 for Basketball Stats
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBA.csv"
spark.sparkContext.addFile(url)

NBAstats = spark.read.option("header", "true").csv(SparkFiles.get("NBA.csv"), inferSchema=True, sep=",")
# NBAstats.show(10)

In [ ]:
NBAstats.count()

In [ ]:
# Print our schema
NBAstats.printSchema()

In [ ]:
from pyspark import SparkFiles
# Connecting to S3 for NBA draft data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBAdraft.csv"
spark.sparkContext.addFile(url)

NBAdraft = spark.read.option("header", "true").csv(SparkFiles.get("NBAdraft.csv"), inferSchema=True, sep=",")
# NBAdraft.show(10)

In [ ]:
NBAdraft.count()

In [ ]:
# Print our schema
NBAdraft.printSchema()

In [12]:
from pyspark import SparkFiles
# Connecting to S3 for Football data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/Football.csv"
spark.sparkContext.addFile(url)

Football = spark.read.option("header", "true").csv(SparkFiles.get("Football.csv"), inferSchema=True, sep=",")
Football.show(10)

+---------+---+----+--------------------+----------------+-----------+--------+-------------+---------------------+----------+----------+------------+----------------+---------------+
|DraftYear|Rnd|Pick|                Team|          Player|GradeFileID|Position|      College|CollegeBudgetFileName|Conference|  latitude|   longitude|ExpensesFootball|RevenueFootball|
+---------+---+----+--------------------+----------------+-----------+--------+-------------+---------------------+----------+----------+------------+----------------+---------------+
|     2015|  1|   1|Tampa Bay Buccaneers|  Jameis Winston|       9434|      QB|  Florida St.| Florida State Uni...|       ACC|30.4404775| -84.2908283|        68893857|       68893857|
|     2015|  1|   2|    Tennessee Titans|  Marcus Mariota|       9435|      QB|       Oregon| University of Oregon|    Pac-12|44.0637862|-123.0787724|        33024841|       72146320|
|     2015|  1|   3|Jacksonville Jaguars|    Dante Fowler|       9436|     OLB| 

In [13]:
Football.count()

1528

In [14]:
# Print our schema
Football.printSchema()

root
 |-- DraftYear: integer (nullable = true)
 |-- Rnd: integer (nullable = true)
 |-- Pick: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- GradeFileID: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- College: string (nullable = true)
 |-- CollegeBudgetFileName: string (nullable = true)
 |-- Conference: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- ExpensesFootball: integer (nullable = true)
 |-- RevenueFootball: integer (nullable = true)



In [ ]:
from pyspark import SparkFiles
# Connecting to S3 for School Data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/SchoolCleaned.csv"
spark.sparkContext.addFile(url)

School_df = spark.read.option("header", "true").csv(SparkFiles.get("SchoolCleaned.csv"), inferSchema=True, sep=",")
School_df.show(10)

In [ ]:
School_df.count()

In [ ]:
# Print our schema
School_df.printSchema()

In [4]:
from pyspark import SparkFiles
# Connecting to S3 for all Basketball data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/Basketball.csv"
spark.sparkContext.addFile(url)

Basketball = spark.read.option("header", "true").csv(SparkFiles.get("Basketball.csv"), inferSchema=True, sep=",")
Basketball.show(10)

+------------------+------+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+----+---+---+---+----+----+-------------+--------------------+--------------------+----------+------------+----------------------+---------------------+
|            player|  team| gp| mpg|fgm| fga|fg_pct|threepm|threepa|three_pct|ftm|fta|ft_pct|tov| pf|orb|drb| rpg|apg|spg|bpg| ppg|pick|   conference|             college|   CollegeBudgetFile|  latitude|   longitude|ExpensesMensBasketball|RevenueMensBasketball|
+------------------+------+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+----+---+---+---+----+----+-------------+--------------------+--------------------+----------+------------+----------------------+---------------------+
|       Chuma Okeke|    AU| 38|29.1|4.4| 8.9| 0.496|    1.4|    3.7|    0.387|1.7|2.4| 0.722|1.7|1.9|2.8|4.1| 6.8|1.9|1.8|1.2|12.0|  16|          SEC|              Auburn|   Auburn University|32.5909562| -85.492978

In [5]:
Basketball.count()

212

In [6]:
Basketball.printSchema()

root
 |-- player: string (nullable = true)
 |-- team: string (nullable = true)
 |-- gp: integer (nullable = true)
 |-- mpg: double (nullable = true)
 |-- fgm: double (nullable = true)
 |-- fga: double (nullable = true)
 |-- fg_pct: double (nullable = true)
 |-- threepm: double (nullable = true)
 |-- threepa: double (nullable = true)
 |-- three_pct: double (nullable = true)
 |-- ftm: double (nullable = true)
 |-- fta: double (nullable = true)
 |-- ft_pct: double (nullable = true)
 |-- tov: double (nullable = true)
 |-- pf: double (nullable = true)
 |-- orb: double (nullable = true)
 |-- drb: double (nullable = true)
 |-- rpg: double (nullable = true)
 |-- apg: double (nullable = true)
 |-- spg: double (nullable = true)
 |-- bpg: double (nullable = true)
 |-- ppg: double (nullable = true)
 |-- pick: integer (nullable = true)
 |-- conference: string (nullable = true)
 |-- college: string (nullable = true)
 |-- CollegeBudgetFile: string (nullable = true)
 |-- latitude: double (nullable = t

**Transformation**

In [ ]:
pdf = NBAstats.toPandas()
df_dedupe = pdf.rename(columns={'FG%':'FG_pct', '3PM':'ThreePM',
                                '3PA':'ThreePA', '3P%':'Three_pct',
                                'FT%':'FT_pct'}).drop_duplicates(subset=['Player'], keep='last').round(3)
NBAstats = spark.createDataFrame(df_dedupe)
NBAstats.show()

In [ ]:
NBAstats.count()

In [ ]:
pdf = NBAdraft.toPandas()
df_dedupe = pdf.rename(columns={'Pk':'Pick','Player':'Name'}).drop_duplicates(subset=['Name'], keep='last')
NBAdraft = spark.createDataFrame(df_dedupe)
NBAdraft.show()

In [ ]:
NBAdraft.count()

In [15]:
pdf = Football.toPandas()
df_dedupe = pdf.rename(columns={'ExpensesFootball':'Expense','RevenueFootball':'Revenue'}).drop_duplicates(subset=['Player'], keep='last')
Football = spark.createDataFrame(df_dedupe)
Football.show()

+---------+---+----+--------------------+----------------+-----------+--------+-------------+---------------------+----------+----------+------------+--------+---------+
|DraftYear|Rnd|Pick|                Team|          Player|GradeFileID|Position|      College|CollegeBudgetFileName|Conference|  latitude|   longitude| Expense|  Revenue|
+---------+---+----+--------------------+----------------+-----------+--------+-------------+---------------------+----------+----------+------------+--------+---------+
|     2015|  1|   1|Tampa Bay Buccaneers|  Jameis Winston|       9434|      QB|  Florida St.| Florida State Uni...|       ACC|30.4404775| -84.2908283|68893857| 68893857|
|     2015|  1|   2|    Tennessee Titans|  Marcus Mariota|       9435|      QB|       Oregon| University of Oregon|    Pac-12|44.0637862|-123.0787724|33024841| 72146320|
|     2015|  1|   3|Jacksonville Jaguars|    Dante Fowler|       9436|     OLB|      Florida| University of Flo...|       SEC|29.6494246| -82.3399209|

In [8]:
pdf = Basketball.toPandas()
df_dedupe = pdf.rename(columns={'ExpensesMensBasketball':'Expense','RevenueMensBasketball':'Revenue'}).drop_duplicates(subset=['player'], keep='last')
Basketball = spark.createDataFrame(df_dedupe)
Basketball.show()

+--------------------+------+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+----+---+---+---+----+----+-------------+--------------------+--------------------+----------+------------+--------+--------+
|              player|  team| gp| mpg|fgm| fga|fg_pct|threepm|threepa|three_pct|ftm|fta|ft_pct|tov| pf|orb|drb| rpg|apg|spg|bpg| ppg|pick|   conference|             college|   CollegeBudgetFile|  latitude|   longitude| Expense| Revenue|
+--------------------+------+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+----+---+---+---+----+----+-------------+--------------------+--------------------+----------+------------+--------+--------+
|         Chuma Okeke|    AU| 38|29.1|4.4| 8.9| 0.496|    1.4|    3.7|    0.387|1.7|2.4| 0.722|1.7|1.9|2.8|4.1| 6.8|1.9|1.8|1.2|12.0|  16|          SEC|              Auburn|   Auburn University|32.5909562| -85.4929785|47846261|95157216|
|     Jerome Robinson|    BC| 35|36.0|7.1|14.7| 0.48

In [ ]:
School_df = School_df.withColumnRenamed("REV_MEN", "Revenue")
School_df = School_df.withColumnRenamed("EXP_MEN", "Expense")
School_df.show()

**Loading**

In [9]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ncaa-athletics.cr5bt5kg46tf.us-west-1.rds.amazonaws.com:5432/NCAA_Athletics"
config = {"user":"postgres", 
          "password": "group1final", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Writing NBA Stats table into RDS
NBAstats.write.jdbc(url=jdbc_url, table='BballStats', mode=mode, properties=config)

In [ ]:
# Writing NBA draft data into RDS
NBAdraft.write.jdbc(url=jdbc_url, table='NBAdraft', mode=mode, properties=config)

In [11]:
# Writing Basketball data into RDS
Basketball.write.jdbc(url=jdbc_url, table='Basketball', mode=mode, properties=config)

In [ ]:
# Writing NFL draft data into RDS
NFL.write.jdbc(url=jdbc_url, table='NFL', mode=mode, properties=config)

In [ ]:
# Writing Schools table into RDS
School_df.write.jdbc(url=jdbc_url, table='schools', mode=mode, properties=config)

In [16]:
# Writing Football table into RDS
Football.write.jdbc(url=jdbc_url, table='Football', mode=mode, properties=config)